In [1]:
import pandas as pd
import numpy as np
import os 
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

2025-10-24 12:10:04.906794: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-24 12:10:04.925375: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-24 12:10:04.931303: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-24 12:10:04.945658: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 12:10:05.725161: W tensorflow/compiler/tf2

In [2]:
df = pd.read_csv("train_df_features.csv")

In [3]:
print(df.head())

              row_id sequence_type sequence_id  sequence_counter      subject  \
0  SEQ_000091_000000        Target  SEQ_000091                 0  SUBJ_032233   
1  SEQ_000091_000001        Target  SEQ_000091                 1  SUBJ_032233   
2  SEQ_000091_000002        Target  SEQ_000091                 2  SUBJ_032233   
3  SEQ_000091_000003        Target  SEQ_000091                 3  SUBJ_032233   
4  SEQ_000091_000004        Target  SEQ_000091                 4  SUBJ_032233   

       orientation                       behavior       phase  \
0  Seated Straight  Moves hand to target location  Transition   
1  Seated Straight  Moves hand to target location  Transition   
2  Seated Straight  Moves hand to target location  Transition   
3  Seated Straight  Moves hand to target location  Transition   
4  Seated Straight  Moves hand to target location  Transition   

                 gesture     acc_x  ...  tof_5_mean  tof_5_std  tof_5_min  \
0  Above ear - pull hair -4.492188  ...      

In [4]:
imu_cols_acceleration = [column for column in df.columns if column.startswith("acc_")]
imu_cols_rotation = [columns for columns in df.columns if columns.startswith("rot_")]



IMUcolumns = imu_cols_acceleration + imu_cols_rotation
columns_targets_gestures = [
    "Above ear - pull hair",
    'Cheek - pinch skin',
     'Eyebrow - pull hair',
    'Eyelash - pull hair',
    'Forehead - pull hairline',
    'Forehead - scratch',
    'Neck - pinch skin',
    'Neck - scratch',
]  
columns_non_gestures = [
            'Write name on leg',
            'Wave hello',
            'Glasses on/off',
            'Text on phone',
            'Write name in air',
            'Feel around in tray and pull out an object',
            'Scratch knee/leg skin',
            'Pull air toward your face',
            'Drink from bottle/cup',
            'Pinch knee/leg skin'
        ]

df['label_collapsed'] = df['gesture'].apply(
    lambda x: x if x in columns_targets_gestures else 'non_target'
)

n_samples = min(len(df), len(IMUcolumns))
label_col = df['label_collapsed'].iloc[:n_samples].values



In [5]:
X = []
y = []

for seq_id, group in df.groupby("sequence_id"):
    X.append(group[IMUcolumns].values)
    y.append(group['label_collapsed'].iloc[0])  

In [6]:
max_len_sequences = int(np.percentile([len(seq) for seq in X], 95))

X_padded = pad_sequences(X, maxlen=max_len_sequences, dtype='float32', padding='post', truncating='post')

labelEncoder = LabelEncoder()

y_encoded = labelEncoder.fit_transform(y)
y_categorical = to_categorical(y_encoded, num_classes=9)

print(f"X_pad shape: {X_padded.shape}")
print(f"y_Categorical shape: {y_categorical.shape}")

X_pad shape: (7573, 115, 10)
y_Categorical shape: (7573, 9)


In [7]:
def LSTM_RNN_IMU_model(input_shape=(None, 10), num_classes=9):
    inputs = keras.layers.Input(shape = input_shape)

    x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(inputs)
    x = keras.layers.Dropout(0.30)(x)
    x = keras.layers.LSTM(64)(x)
    x = keras.layers.Dropout(0.30)(x)

    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.30)(x)

    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
    model = keras.models.Model(inputs, outputs)

    return model
    

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_padded, y_categorical, test_size=0.20, shuffle = False
)

print(f"X train shape: {X_train.shape}")
print(f"Y train shape: {Y_train.shape}")
print(f"X test shape: {X_test.shape}")
print(f"Y test shape: {Y_test.shape}")

X train shape: (6058, 115, 10)
Y train shape: (6058, 9)
X test shape: (1515, 115, 10)
Y test shape: (1515, 9)


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_padded, y_categorical, test_size=0.20, shuffle = False
)

print(f"X train shape: {X_train.shape}")
print(f"Y train shape: {Y_train.shape}")
print(f"X test shape: {X_test.shape}")
print(f"Y test shape: {Y_test.shape}")

X train shape: (6058, 115, 10)
Y train shape: (6058, 9)
X test shape: (1515, 115, 10)
Y test shape: (1515, 9)


In [10]:
model = LSTM_RNN_IMU_model(input_shape=(max_len_sequences, len(IMUcolumns)))


2025-10-24 12:10:42.041335: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
optimizers = keras.optimizers.AdamW(learning_rate=1e-4, clipnorm=1.0)

model.compile(
    optimizer=optimizers,
    loss="categorical_crossentropy",
    metrics=['accuracy']
)

history = model.fit(
    X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=50
)

model.summary()

Epoch 1/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 30s 141ms/step - accuracy: 0.1244 - loss: 2.4412 - val_accuracy: 0.3498 - val_loss: 2.1346
Epoch 2/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 25s 134ms/step - accuracy: 0.2592 - loss: 2.2035 - val_accuracy: 0.3492 - val_loss: 2.0690
Epoch 3/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step - accuracy: 0.3230 - loss: 2.0953 - val_accuracy: 0.3551 - val_loss: 1.8264
Epoch 4/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 26s 136ms/step - accuracy: 0.3500 - loss: 1.8568 - val_accuracy: 0.3723 - val_loss: 1.7427
Epoch 5/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 25s 133ms/step - accuracy: 0.3754 - loss: 1.7410 - val_accuracy: 0.3855 - val_loss: 1.6824
Epoch 6/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 25s 133ms/step - accuracy: 0.3922 - loss: 1.6996 - val_accuracy: 0.3914 - val_loss: 1.6785
Epoch 7/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 25s 133ms/step - accuracy: 0.3713 - loss: 1.7064 - val_accuracy: 0.3802 - val_loss: 1.6578
Epoch 8/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 26s 137ms/step - accuracy: 0.3840 - loss: 1

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 115, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 115, 256)       │       142,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 115, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        82,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 703,005 (2.68 MB)

 Trainable params: 234,249 (915.04 KB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 468,500 (1.79 MB)

In [13]:
train_loss , train_accuracy = model.evaluate(X_train,Y_train)
test_loss , test_accuracy = model.evaluate(X_test,Y_test)

print(f"The train accuracy {train_accuracy}")
print(f"The test accuracy {test_accuracy}")

model.save("IMU_RNN_LSTM_variables_macrof1.h5")

190/190 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.5398 - loss: 1.2340
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.4575 - loss: 1.3546


The train accuracy 0.5379663109779358
The test accuracy 0.4706270694732666
